<a href="https://colab.research.google.com/github/zhangxs131/Transformers_example/blob/main/%E4%B8%AD%E6%96%87%E5%8F%A5%E5%AD%90%E5%85%B3%E7%B3%BB%E6%8E%A8%E6%96%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#中文句子关系推动

*   model bert-base-chinese
*   dataset ChnSentiCorp中把文本截取40长度，[:20] [20:] 为两个句子作为训练集

In [2]:
!pip install transformers 
!pip install datasets

In [23]:
#加载数据集
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch
from datasets import load_dataset
from transformers import BertTokenizer,BertModel
import random
import transformers
transformers.logging.set_verbosity_error()


class SentiDataset(Dataset):
  def __init__(self,split):
    super().__init__()
    self.dataset=load_dataset(path='seamew/ChnSentiCorp',split=split)
    def fn(data):
      return len(data['text'])>40
    self.dataset.filter(fn)

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,i):

    sent1=self.dataset[i]['text'][:20]
    sent2=self.dataset[i]['text'][20:]
    label=1

    if random.randint(0,1)==0:
      j=random.randint(0,len(self.dataset)-1)
      sent2=self.dataset[j]['text'][20:]
      label=0

    return sent1,sent2,label



    return self.dataset[i]['text']

tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')

def collate_fn(data):
  sents=[i[:2] for i in data]
  labels=[i[2] for i in data]

  data=tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents,
                  truncation=True,
                  padding='max_length',
                  max_length=45,
                  return_tensors='pt',
                  return_length='True',
                  add_special_tokens=True)
  labels=torch.LongTensor(labels)
  return data['input_ids'],data['attention_mask'],data['token_type_ids'],labels

dataloader=DataLoader(dataset=SentiDataset('train'),
           batch_size=16,
           shuffle=True,
           collate_fn=collate_fn,
           drop_last=True)

for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):
  break

print(tokenizer.decode(input_ids[0]))
print(labels[0])

Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-4df94ee7a91f1822.arrow


[CLS] 缺 点 ： 房 间 比 较 老 旧 ， 电 梯 果 然 如 网 友 评 价 非 [SEP] 较 困 难 ， 因 为 [UNK] 盘 芯 片 驱 动 比 较 怪 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor(0)


In [26]:
#模型
from transformers import AdamW
pre=BertModel.from_pretrained('bert-base-chinese')


for param in pre.parameters():
  param.requires_grad_(False)

class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc=torch.nn.Linear(768,2)

  def forward(self,input_ids,attention_mask,token_type_ids,device):
    with torch.no_grad():
      pre.to(device)
      out=pre(input_ids,attention_mask,token_type_ids)

    out=self.fc(out.last_hidden_state[:,0])
    out=out.softmax(dim=1)
    return out

model=Model()
optimizer=AdamW(model.parameters(),lr=5e-4)
criterion=torch.nn.CrossEntropyLoss()
device='cuda' if torch.cuda.is_available() else 'cpu'

def test(epoch):
  total=0
  correct=0

  test_loader=DataLoader(dataset=SentiDataset('validation'),
             batch_size=32,
             collate_fn=collate_fn,
             drop_last=True)

  model.eval().to(device)
  with torch.no_grad():
    for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(test_loader):
      input_ids=input_ids.to(device)
      attention_mask=attention_mask.to(device)
      token_type_ids=token_type_ids.to(device)
      labels=labels.to(device)

      out=model(input_ids,attention_mask,token_type_ids,device)
      out=out.argmax(dim=1)
      total+=len(labels)
      correct+=(out==labels).sum().item()

  print('Epoch {} Accuarcy {}'.format(epoch,correct/total))

def train(Epochs):
  for epoch in range(Epochs):
    model.train().to(device)
    
    for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):
      input_ids=input_ids.to(device)
      attention_mask=attention_mask.to(device)
      token_type_ids=token_type_ids.to(device)
      labels=labels.to(device)

      out=model(input_ids,attention_mask,token_type_ids,device)

      loss=criterion(out,labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if i%50==0:
        out=out.argmax(dim=1)
        accuarcay=(out==labels).sum().item()/len(labels)

        print('Epoch {} Step {}, Loss {} Accuarcy {}'.format(epoch+1,i,loss.item(),accuarcay))

    test(epoch+1)
#训练3轮
train(3)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch 1 Step 0, Loss 0.6742010116577148 Accuarcy 0.5625
Epoch 1 Step 50, Loss 0.5527991652488708 Accuarcy 0.6875
Epoch 1 Step 100, Loss 0.3861014246940613 Accuarcy 0.9375
Epoch 1 Step 150, Loss 0.5343166589736938 Accuarcy 0.8125
Epoch 1 Step 200, Loss 0.3891306221485138 Accuarcy 0.9375
Epoch 1 Step 250, Loss 0.3943360149860382 Accuarcy 0.9375
Epoch 1 Step 300, Loss 0.3377110958099365 Accuarcy 1.0
Epoch 1 Step 350, Loss 0.4379882514476776 Accuarcy 0.875
Epoch 1 Step 400, Loss 0.3516303300857544 Accuarcy 0.9375
Epoch 1 Step 450, Loss 0.6172651052474976 Accuarcy 0.6875
Epoch 1 Step 500, Loss 0.4706061780452728 Accuarcy 0.875
Epoch 1 Step 550, Loss 0.4764832854270935 Accuarcy 0.875


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-04e37f05838e9029.arrow


Epoch 1 Accuarcy 0.8648648648648649
Epoch 2 Step 0, Loss 0.42543506622314453 Accuarcy 0.875
Epoch 2 Step 50, Loss 0.32489830255508423 Accuarcy 1.0
Epoch 2 Step 100, Loss 0.4281346797943115 Accuarcy 0.9375
Epoch 2 Step 150, Loss 0.4291769862174988 Accuarcy 0.875
Epoch 2 Step 200, Loss 0.5301893353462219 Accuarcy 0.75
Epoch 2 Step 250, Loss 0.4140092730522156 Accuarcy 0.9375
Epoch 2 Step 300, Loss 0.47253552079200745 Accuarcy 0.8125
Epoch 2 Step 350, Loss 0.5089864134788513 Accuarcy 0.75
Epoch 2 Step 400, Loss 0.6019094586372375 Accuarcy 0.6875
Epoch 2 Step 450, Loss 0.4175141155719757 Accuarcy 0.875
Epoch 2 Step 500, Loss 0.5147053599357605 Accuarcy 0.8125
Epoch 2 Step 550, Loss 0.4840274453163147 Accuarcy 0.8125


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-04e37f05838e9029.arrow


Epoch 2 Accuarcy 0.8665540540540541
Epoch 3 Step 0, Loss 0.3402426242828369 Accuarcy 1.0
Epoch 3 Step 50, Loss 0.43492990732192993 Accuarcy 0.875
Epoch 3 Step 100, Loss 0.4525097906589508 Accuarcy 0.875
Epoch 3 Step 150, Loss 0.3622138798236847 Accuarcy 0.9375
Epoch 3 Step 200, Loss 0.49160781502723694 Accuarcy 0.8125
Epoch 3 Step 250, Loss 0.5142028331756592 Accuarcy 0.8125
Epoch 3 Step 300, Loss 0.45612552762031555 Accuarcy 0.8125
Epoch 3 Step 350, Loss 0.3769385516643524 Accuarcy 0.9375
Epoch 3 Step 400, Loss 0.45309191942214966 Accuarcy 0.875
Epoch 3 Step 450, Loss 0.4848352074623108 Accuarcy 0.8125
Epoch 3 Step 500, Loss 0.48708397150039673 Accuarcy 0.8125
Epoch 3 Step 550, Loss 0.3610212206840515 Accuarcy 0.9375


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-04e37f05838e9029.arrow


Epoch 3 Accuarcy 0.8724662162162162
